In [13]:
import os
import shutil
import subprocess

import numpy as np

from hyperopt import hp
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

import tensorflow as tf
import keras
from keras import backend as K
from keras.datasets import mnist
from keras.layers import BatchNormalization, Flatten, Dense
from keras.layers import Input, Conv2D, Convolution2D
from keras.layers import Activation, concatenate, Dropout
from keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.utils import np_utils, to_categorical
from keras.applications.imagenet_utils import _obtain_input_shape

import Augmentor
import json

import tuner
from tuner import utils
from tuner import load_data
from tuner import augment_data
from tuner import use_hyperas
from tuner import net
from tuner.dataset import ClassificationDataset, AugmentDataset

from datetime import datetime

In [3]:
from easydict import EasyDict
params = EasyDict({
    'src_dir': '../micin-dataset/eyes',
    'dst_dir': './brain',
    'bs': 32,
    'epochs': 10,
    'teaching_file': 'label.tsv',
    'save_model_file': 'tuner.{}.model.hdf5'.format(int(datetime.now().timestamp()))
})
try:
    __file__
    import argparse
    argparse = argparse.ArgumentParser()
    parser.add_argument('-d','--dataset', dest='src_dir', type=str,
                         help='dataset directory')
    parser.add_argument('-t','--teaching-file', dest='teaching_file', type=str, default=None, 
                         help='teaching file. csv or tsv')
    parser.add_argument('-o','--output', dest='save_model_file', type=str, default=params.save_model_file,
                         help='dataset directory')
    parser.add_argument('-b','--batchsize', dest='bs', type=int, default=params.bs,
                         help='batch size')
    parser.add_argument('-e','--epochs', type=int, default=params.epochs,
                         help='epochs')
    args = parser.parse_args()
except:
    args = params


### Convert  dataset from brain-dir to ready-dir 

In [ ]:
tmp_dir = 'tmp'

In [4]:
df = load_data.df_fromdir_eyes(args.src_dir, args.teaching_file)

In [5]:
load_data.classed_dir_fromdf(df, tmp_dir)

### Load dataset via ready-dir

In [6]:
brain = ClassificationDataset(tmp_dir)

In [7]:
brain = AugmentDataset(brain)

In [8]:
brain.augmented_dir

'/home/fytroo/Tuner/dev/standard_datasets/5a670683b037cb562cc8c116/auged'

In [9]:
brain.dataset.train_dir

'/home/fytroo/Tuner/dev/standard_datasets/5a670683b037cb562cc8c116/train'

### Search best condition of data augmentation

In [10]:
brain.search_opt_augment(model=net.neoaug)

def data():
    train_dir = '/home/fytroo/Tuner/dev/standard_datasets/5a670683b037cb562cc8c116/train'
    test_dir = '/home/fytroo/Tuner/dev/standard_datasets/5a670683b037cb562cc8c116/validation'
    resize = '96'
    rescale = '1'
    df = load_data.df_fromdir_classed(train_dir)
    x_train, y_train = load_data.load_fromdf(df, resize=resize, rescale=rescale)
    df = load_data.df_fromdir_classed(test_dir)
    x_test, y_test = load_data.load_fromdf(df, resize=resize, rescale=rescale)

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    return x_train, y_train, x_test, y_test

            name label             handle  \
0   101_0179.JPG  異常なし  異常なし/101_0179.JPG   
1   111_0148.JPG  異常なし  異常なし/111_0148.JPG   
2   108_0110.JPG  異常なし  異常なし/108_0110.JPG   
3   125_0080.JPG  異常なし  異常なし/125_0080.JPG   
4   198_0119.JPG  異常なし  異常なし/198_0119.JPG   
5       MS83.jpg    MS        MS/MS83.jpg   
6      MS122.jpg    MS       MS/MS122.jpg   
7      MS164.jpg    MS       M

--------------------------------------------------------------------------------
Operations: 4
	0: Flip (probability=0.5 top_bottom_left_right=LEFT_RIGHT )
	1: CropPercentage (probability=1 percentage_area=0.8 centre=False randomise_percentage_area=False )
	2: Resize (probability=1 width=96 height=96 resample_filter=BICUBIC )
	3: Shear (probability=0.3 max_shear_left=2 max_shear_right=2 )
Images: 0
Classes: 0

You can remove operations using the appropriate index and the remove_operation(index) function.
Epoch 1/10
 - 4s - loss: 0.9867 - acc: 0.5675 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 2/10
 - 1s - loss: 0.8783 - acc: 0.5917 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 3/10
 - 1s - loss: 0.8628 - acc: 0.5554 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 4/10
 - 1s - loss: 0.8412 - acc: 0.5998 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 5/10
 - 1s - loss: 0.8084 - acc: 0.6250 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 6/10
 - 1s - loss: 0.8305 - acc: 0.5958 - val_loss: 10.7454 - va

Epoch 2/10
 - 1s - loss: 0.8480 - acc: 0.6159 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 3/10
 - 1s - loss: 0.8719 - acc: 0.5625 - val_loss: 9.2168 - val_acc: 0.3667
Epoch 4/10
 - 1s - loss: 0.8336 - acc: 0.6169 - val_loss: 10.0997 - val_acc: 0.3333
Epoch 5/10
 - 1s - loss: 0.7941 - acc: 0.6220 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 6/10
 - 1s - loss: 0.8114 - acc: 0.6179 - val_loss: 9.4555 - val_acc: 0.4000
Epoch 7/10
 - 1s - loss: 0.8256 - acc: 0.6169 - val_loss: 9.5496 - val_acc: 0.3667
Epoch 8/10
 - 1s - loss: 0.8066 - acc: 0.6169 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 9/10
 - 1s - loss: 0.8255 - acc: 0.6028 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 10/10
 - 1s - loss: 0.8147 - acc: 0.6099 - val_loss: 9.7953 - val_acc: 0.3333
Test accuracy: 0.333333343267
--------------------------------------------------------------------------------
Operations: 2
	0: Flip (probability=0.5 top_bottom_left_right=LEFT_RIGHT )
	1: Shear (probability=0.3 max_shear_left=2 max_shear_right=2

### Execute data augmentation with best condition

In [11]:
brain.augment_dataset()

Processing 178_0112.JPG:   2%|▏         | 3/184 [00:00<00:07, 24.75 Samples/s]

Initialised with 535 image(s) found.
Output directory set to /home/fytroo/Tuner/dev/standard_datasets/5a670683b037cb562cc8c116/train/異常なし/output.

Processing MS070.jpg:   5%|▌         | 10/184 [00:00<00:02, 62.86 Samples/s]    

Initialised with 46 image(s) found.
Output directory set to /home/fytroo/Tuner/dev/standard_datasets/5a670683b037cb562cc8c116/train/MS/output.

Processing PD044.jpg:   3%|▎         | 5/184 [00:00<00:05, 32.83 Samples/s]  

...

Initialised with 226 image(s) found.
Output directory set to /home/fytroo/Tuner/dev/standard_datasets/5a670683b037cb562cc8c116/train/異常あり/output.

Processing N028.jpg:   9%|▉         | 17/184 [00:00<00:01, 85.18 Samples/s]     

Initialised with 79 image(s) found.
Output directory set to /home/fytroo/Tuner/dev/standard_datasets/5a670683b037cb562cc8c116/train/N/output.

Processing PS107.jpg:   3%|▎         | 5/184 [00:00<00:05, 34.04 Samples/s] 

Initialised with 75 image(s) found.
Output directory set to /home/fytroo/Tuner/dev/standard_datasets/5a670683b037cb562cc8c116/train/PS/output.

### Tuning hyper parameter of CNN

In [14]:
best_condition, best_model = use_hyperas.exec_hyperas(
    brain.augmented_dir,
    brain.dataset.validation_dir,
    net.simplenet,
    batch_size = 32,
    epochs=10,
    optimizer='adam',
    rescale=1./255
)
best_model.save(args.save_model_file)
shutil.rmtree(tmp_dir)

def data():
    train_dir = '/home/fytroo/Tuner/dev/standard_datasets/5a670683b037cb562cc8c116/auged'
    test_dir = '/home/fytroo/Tuner/dev/standard_datasets/5a670683b037cb562cc8c116/validation'
    resize = '96'
    rescale = '0.00392156862745098'
    df = load_data.df_fromdir_classed(train_dir)
    x_train, y_train = load_data.load_fromdf(df, resize=resize, rescale=rescale)
    df = load_data.df_fromdir_classed(test_dir)
    x_test, y_test = load_data.load_fromdf(df, resize=resize, rescale=rescale)

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    return x_train, y_train, x_test, y_test

            name label             handle  \
0   101_0179.JPG  異常なし  異常なし/101_0179.JPG   
1   111_0148.JPG  異常なし  異常なし/111_0148.JPG   
2   108_0110.JPG  異常なし  異常なし/108_0110.JPG   
3   125_0080.JPG  異常なし  異常なし/125_0080.JPG   
4   198_0119.JPG  異常なし  異常なし/198_0119.JPG   
5       MS83.jpg    MS        MS/MS83.jpg   
6      MS122.jpg    MS       MS/MS122.jpg   
7      MS164

Epoch 7/10
 - 1s - loss: 0.7125 - acc: 0.6784 - val_loss: 1.0111 - val_acc: 0.5333
Epoch 8/10
 - 1s - loss: 0.5517 - acc: 0.7337 - val_loss: 1.1606 - val_acc: 0.5333
Epoch 9/10
 - 1s - loss: 0.4213 - acc: 0.7880 - val_loss: 1.4117 - val_acc: 0.4667
Epoch 10/10
 - 1s - loss: 0.3440 - acc: 0.8243 - val_loss: 1.4810 - val_acc: 0.5667
Test accuracy: 0.566666662693
Train on 1104 samples, validate on 30 samples
Epoch 1/10
 - 1s - loss: 1.2996 - acc: 0.3406 - val_loss: 1.1880 - val_acc: 0.3333
Epoch 2/10
 - 1s - loss: 1.1724 - acc: 0.3714 - val_loss: 1.1493 - val_acc: 0.3333
Epoch 3/10
 - 1s - loss: 1.1182 - acc: 0.4221 - val_loss: 1.1692 - val_acc: 0.4000
Epoch 4/10
 - 1s - loss: 1.0150 - acc: 0.5181 - val_loss: 1.1730 - val_acc: 0.4000
Epoch 5/10
 - 1s - loss: 0.9196 - acc: 0.5707 - val_loss: 1.1626 - val_acc: 0.3667
Epoch 6/10
 - 1s - loss: 0.7293 - acc: 0.6712 - val_loss: 1.3175 - val_acc: 0.5000
Epoch 7/10
 - 1s - loss: 0.5534 - acc: 0.7400 - val_loss: 1.1415 - val_acc: 0.5333
Epoch 8/10

KeyboardInterrupt: 